# Reimplementing basics of tensorflow
Exersise to understand how tensorflow is built, again based on the book by Chollet. In the 2021 book version there are some coding issues on page 63. Corrected in this code. 

In [2]:
import tensorflow as tf

In [3]:
class NaiveDense:
    """
    A basic dense layer, requires an activation function
    """
    def __init__(self, input_size, output_size, activation):
        self.activation = activation
        w_shape = (input_size, output_size) # weights tuple shape
        w_initial_vals = tf.random.uniform(w_shape, minval=0, maxval=0.1)
        self.W = tf.Variable(w_initial_vals)
        
        b_shape = (output_size,) # obviously a vector not a matrix output
        b_intial_vals = tf.zeros(b_shape)
        self.b = tf.Variable(b_intial_vals)
    
    def  __call__(self,inputs):
        return self.activation(tf.matmul(inputs, self.W)+ self.b)
    
    @property # not used to this decorator
    def weights(self):
        return [self.W, self.b]
        

In [4]:
class NaiveSequential:
    
    def __init__(self, layers):
        self.layers = layers
    
    def __call__(self, inputs):
        x = inputs
        for layer in self.layers:
            x = layer(x)
        return x
    
    @property
    def weights(self):
        weights = []
        for layer in self.layers:
            weights += layer.weights
        return weights

The sequential just applys each layer in sequence. 

In [5]:
model = NaiveSequential([NaiveDense(28*28, 512, tf.nn.relu),NaiveDense(512, 10, tf.nn.softmax)])

In [6]:
assert len(model.weights) == 4

Seems pretty simple to create custom neural nets obivously the speed disadvantage ultimately makes this solution not worthwhile. 

In [7]:
import math

class BatchGenerator:
    def __init__(self, images, labels, batch_size = 128):
        assert len(images) == len(labels) # should probably be a raise exception
        self.index = 0
        self.images = images
        self.labels = labels
        self.batch_size = batch_size
        self.num_batches = math.ceil(len(images)/batch_size)
        
    def next(self):
        images = self.images[self.index : self.index + self.batch_size]
        labels = self.labels[self.index : self.index + self.batch_size]
        self.index += self.batch_size
        return images, labels

In [8]:
def one_training_step(model, images_batch, labels_batch):
    with tf.GradientTape() as tape:
        predictions = model(images_batch)
        per_sample_losses = tf.keras.losses.sparse_categorical_crossentropy(labels_batch, predictions)
        aver_loss = tf.reduce_mean(per_sample_losses)
    gradients = tape.gradient(aver_loss, model.weights)
    update_weights(gradients, model.weights)
    return aver_loss

In [9]:
learning_rate = 1e-3

def update_weights(gradients, weights):
    for grad, weight in zip(gradients, weights):
        weight.assign_sub(grad*learning_rate)
        


In [12]:
def fit(model, images, labels, epochs, batch_size=128):
    for epoch_counter in range(epochs):
        print(f"Epoch # {epoch_counter}")
        batch_gen = BatchGenerator(images, labels)
        for batch_counter in range(batch_gen.num_batches):
            images_batch, labels_batch = batch_gen.next()
            loss = one_training_step(model, images_batch, labels_batch)
            if batch_counter % 100 == 0:
                print(f"loss at batch #{batch_counter}:{loss:.2f}")
                

In [13]:
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images,test_labels) = mnist.load_data()
train_images = train_images.reshape((60000,28*28)) 
train_images = train_images.astype("float32")/255  
test_images = test_images.reshape((10000,28*28)) 
test_images = test_images.astype("float32")/255 
fit(model, train_images, train_labels, 10)


Epoch # 0
loss at batch #0:4.45
loss at batch #100:2.28
loss at batch #200:2.25
loss at batch #300:2.11
loss at batch #400:2.26
Epoch # 1
loss at batch #0:1.94
loss at batch #100:1.90
loss at batch #200:1.86
loss at batch #300:1.73
loss at batch #400:1.85
Epoch # 2
loss at batch #0:1.61
loss at batch #100:1.60
loss at batch #200:1.53
loss at batch #300:1.44
loss at batch #400:1.52
Epoch # 3
loss at batch #0:1.35
loss at batch #100:1.35
loss at batch #200:1.25
loss at batch #300:1.22
loss at batch #400:1.28
Epoch # 4
loss at batch #0:1.15
loss at batch #100:1.17
loss at batch #200:1.05
loss at batch #300:1.05
loss at batch #400:1.11
Epoch # 5
loss at batch #0:1.00
loss at batch #100:1.03
loss at batch #200:0.91
loss at batch #300:0.93
loss at batch #400:0.99
Epoch # 6
loss at batch #0:0.88
loss at batch #100:0.92
loss at batch #200:0.81
loss at batch #300:0.84
loss at batch #400:0.90
Epoch # 7
loss at batch #0:0.80
loss at batch #100:0.83
loss at batch #200:0.73
loss at batch #300:0.77


Wow so slow!!!! Glad keras is optimised.